In [ ]:
#Alejandro García Girón
#Pegamos estos comandos para abrir un entorno spark en la nube

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

. Inicializar y cargar el contexto spark
2. Número de países distintos en cada dataset. Coinciden?
3. Total de deuda contraida por cada pais: total amount of debt (in USD) DT.AMT.MLAT.CD
4. Media de los indicadores de deuda (tabla uno): DT.AMT.BLAT.CD, DT.DIS.BLAT.CD, DT.INT.BLAT.CD
5. Los 20 paises con DT.AMT.DLXF.CD más alto
6. Pais con los datos informados todos los años.
7. Media anual de los distintos indicadores de desarrollo
8. Podrías decirme el total de deuda acumulada DT.AMT.MLAT.CD por los 10 países con un valor en media menor de SP.DYN.CBRT.IN¶
9. ¿Hay alguna relación entre los nacimientos y el indicador DT.AMT.DLXF.CD? ¿Cómo podrías demostrarlo?

In [ ]:
#Cargamos los datos


In [ ]:
deuda = sc.textFile('indicadores_deuda.csv')
desarrollo = sc.textFile('indicadores_desarrollo.csv')

In [ ]:
deuda.count() 

125

Tratamiento de los datos

In [ ]:
deuda.max()

'country_name,country_code,DT.AMT.BLAT.CD,DT.AMT.DLXF.CD,DT.AMT.DPNG.CD,DT.AMT.MLAT.CD,DT.AMT.OFFT.CD,DT.AMT.PBND.CD,DT.AMT.PCBK.CD,DT.AMT.PROP.CD,DT.AMT.PRVT.CD,DT.DIS.BLAT.CD,DT.DIS.DLXF.CD,DT.DIS.MLAT.CD,DT.DIS.OFFT.CD,DT.DIS.PCBK.CD,DT.DIS.PROP.CD,DT.DIS.PRVT.CD,DT.INT.BLAT.CD,DT.INT.DLXF.CD,DT.INT.DPNG.CD,DT.INT.MLAT.CD,DT.INT.OFFT.CD,DT.INT.PBND.CD,DT.INT.PCBK.CD,DT.INT.PROP.CD,DT.INT.PRVT.CD'

In [ ]:
#Parseado de los datos y limpieza

#Separamos las bases de datos por comas ','
alfa_deuda = deuda.map(lambda x :x.split(","))
alfa_desarrollo = desarrollo.map(lambda x: x.split(","))     



In [ ]:
headerout = alfa_desarrollo.first() # quitamos encabezados de desarrollo                                           
beta_desarrollo = alfa_desarrollo.filter(lambda x : x != headerout) 

In [ ]:
headerout1 = alfa_deuda.first()                                            
beta_deuda = alfa_deuda.filter(lambda x : x != headerout1) #quitamos los encabezados, darán problemas

In [ ]:
beta_desarrollo.take(7) #vemos que lo anterior se ejecuto bien

In [ ]:
#mecanismo par cerciorarnos de las longitudes de las bbdd

Mecanismo par cerciorarnos de las longitudes de las bbdd

In [ ]:
beta_desarrollo.map(lambda x: (len(x))).distinct().count() 

2

In [ ]:
beta_deuda.map(lambda x: (len(x))).distinct().count() 

2

In [ ]:
 # vemos el tamño por fila de los primeros registros
beta_deuda.map(lambda x: (len(x),x[0:2])).take(5)    

[(27, ['Afghanistan', 'AFG']),
 (27, ['Albania', 'ALB']),
 (27, ['Algeria', 'DZA']),
 (27, ['Angola', 'AGO']),
 (27, ['Armenia', 'ARM'])]

In [ ]:
#idem cambiando la bbdd
beta_desarrollo.map(lambda x: (len(x),x[0:2])).take(5)    

[(6, ['Albania', 'ALB']),
 (6, ['Albania', 'ALB']),
 (6, ['Albania', 'ALB']),
 (6, ['Australia', 'AUS']),
 (6, ['Australia', 'AUS'])]

In [ ]:
#Podemos observar los registros de tamaño 6
beta_desarrollo.map(lambda x: (len(x),x)).filter(lambda x: x[0] == 6).take(10)

[(6,
  ['Albania',
   'ALB',
   '2014 [YR2014]',
   '1023040800000',
   '73.3202469198429',
   '12.259']),
 (6,
  ['Albania',
   'ALB',
   '2015 [YR2015]',
   '1145500940000',
   '79.8644448747259',
   '12.197']),
 (6,
  ['Albania',
   'ALB',
   '2016 [YR2016]',
   '1188814490000',
   '80.7355748396923',
   '12.08']),
 (6,
  ['Australia',
   'AUS',
   '2014 [YR2014]',
   '674700000',
   '0.042207528166503',
   '13.2']),
 (6,
  ['Australia',
   'AUS',
   '2015 [YR2015]',
   '762718000',
   '0.0469480198522591',
   '12.9']),
 (6,
  ['Australia',
   'AUS',
   '2016 [YR2016]',
   '907555000',
   '0.054648482520169',
   '12.9']),
 (6,
  ['Barbados',
   'BRB',
   '2014 [YR2014]',
   '12175704025',
   '129.629591844133',
   '10.916']),
 (6,
  ['Barbados',
   'BRB',
   '2015 [YR2015]',
   '13365027241',
   '141.413895259761',
   '10.806']),
 (6,
  ['Belarus',
   'BLR',
   '2014 [YR2014]',
   '19781184418',
   '24.5487262642116',
   '12.5']),
 (6,
  ['Belarus',
   'BLR',
   '2015 [YR2015]',
   

In [ ]:
#recoemendacion de crear un vector de dimensiones por si más adelante es preciso aportar la proporcion de las estucturas
longit_dll = beta_desarrollo.map(lambda x: (len(x), x))  

In [ ]:
longit_deu = beta_deuda.map(lambda x: (len(x), x))

In [ ]:
#Creamos una función 

In [ ]:
#definimos función donde si la linea es igual a 6 nos de la linea 1 
def parseadito(line):                  
    if line[0] == 6:
        return line[1]
    else:
        l = line[1][1:]
        return   l.insert(0, line[1][0]+line[1][1])  #no lo termine de entender bien

In [ ]:
def parseadoto(line):              #revisar esto bien    
    if line[0] == 27:
        return line[1]
    else:
        l = line[1][1:]
        return   l.insert(0, line[1][0]+line[1][1])

In [ ]:
#Tamaño no siendo 6 las filtramos
filter_6 = beta_desarrollo.filter(lambda x: (len(x) != 6))           

In [ ]:
#solo con los primeros vals
filter_6 = filter_6.map(lambda x: (x[0],x[2],x[3],x[4],x[5],x[6]))   

In [ ]:
filtro_six = beta_desarrollo.filter(lambda x: (len(x)== 6))             

In [ ]:
#unimos las de 6 con las que solo nos habiamos queddo los primeros valores
clean_beta_desarrollo = filter_6.union(filtro_six)                    

In [ ]:
#nos cercioramos del proceso
clean_beta_desarrollo.take(1)                                       

[('"Bahamas',
  'BHS',
  '2014 [YR2014]',
  '5158420000',
  '47.2672793747079',
  '14.34')]

In [ ]:
#comprobamos tamaño

In [ ]:
clean_beta_desarrollo.map(lambda x: (len(x))).distinct().count()   

1

In [ ]:
#Ahora el mismo proceso pero con la bese deuda

In [ ]:
filter6Deu = beta_deuda.filter(lambda x: (len(x) != 27))                

In [ ]:
filter6deuAlfa = filter6Deu.map(lambda x: (x[0],x[2:],x[3],x[4],x[5],x[6],x[7],x[8],x[9],x[10],x[11],x[12],x[13],x[14],x[15],x[16],x[17],x[18],x[19],x[20],x[21],x[22],x[23],x[24],x[25],x[26],x[27]))

In [ ]:
filtro6Omega = beta_deuda.filter(lambda x: (len(x)== 27))

In [ ]:
clean_beta_deuda = filtro6Omega.union(filter6deuAlfa) #unimos

In [ ]:
clean_beta_deuda.take(1)

[['Afghanistan',
  'AFG',
  '61739336.9',
  '100847181.9',
  '',
  '39107845',
  '100847181.9',
  '',
  '',
  '',
  '',
  '49114729.4',
  '72894453.7',
  '23779724.3',
  '72894453.7',
  '',
  '',
  '',
  '39903620.1',
  '53239440.1',
  '',
  '13335820',
  '53239440.1',
  '',
  '',
  '',
  '']]

In [ ]:
#nos cercioramos de su longitud

In [ ]:
#el bloque de datos se estructura como uno. perfecto.
clean_beta_deuda.map(lambda x: (len(x))).distinct().count()                

1

pregunta 2
Número de países distintos en cada dataset. Coinciden?
Solicitamos el numero de valores distintos que hay en la primera columna

In [ ]:
#pedimos valores de cada uno
deuda_soberana = clean_beta_deuda.map(lambda x : x[0]).distinct()                  
desarrollo_pais = clean_beta_desarrollo.map(lambda x: x[0]).distinct()                          

In [ ]:
#contamos
deuda_soberana.count(),desarrollo_pais.count()                                      

(123, 51)

In [ ]:
#Listado de paises de la bbdd: paises
desarrollo_pais.distinct().collect()                           

['Belize',
 'Botswana',
 'Guatemala',
 'Hungary',
 'Palau',
 'Russian Federation',
 'Switzerland',
 'Ukraine',
 'United States',
 'Uruguay',
 'Zambia',
 'Belarus',
 'Georgia',
 'Jamaica',
 'Japan',
 'Jordan',
 'Moldova',
 'Namibia',
 'New Zealand',
 'Papua New Guinea',
 'Seychelles',
 'Singapore',
 'Spain',
 'St. Lucia',
 'Turkey',
 'United Kingdom',
 '"Bahamas',
 'Albania',
 'Australia',
 'Barbados',
 'Bhutan',
 'Brazil',
 'El Salvador',
 'Ireland',
 'Kazakhstan',
 'Kyrgyz Republic',
 'Malawi',
 'Peru',
 'Philippines',
 'Solomon Islands',
 'St. Kitts and Nevis',
 'St. Vincent and the Grenadines',
 'Thailand',
 '"Korea',
 '"Micronesia',
 'Colombia',
 'Indonesia',
 'Malaysia',
 'San Marino',
 'Sri Lanka',
 'Tonga']

In [ ]:
#Bahamas sigue con las comillas esas...........



3. Total de deuda contraida por cada pais: total amount of debt (in USD) 

In [ ]:
#Realizamos la agrupacion segundo la primera columna de paises y según DT.AMT.MLAT.CD
deuda_soberana.take(20)
deuda_soberana = clean_beta_deuda.groupBy(lambda x: (x[0],x[5]))      
deuda_soberana.take(20)
#despues del nombre pais sale el monto

[(('Bosnia and Herzegovina', '290170946.5'),
  <pyspark.resultiterable.ResultIterable at 0x7fcb9b236710>),
 (('Brazil', '2514318741.6'),
  <pyspark.resultiterable.ResultIterable at 0x7fcb9b236198>),
 (('Bulgaria', '228993019.8'),
  <pyspark.resultiterable.ResultIterable at 0x7fcb9b221f28>),
 (('Burundi', '16115111.8'),
  <pyspark.resultiterable.ResultIterable at 0x7fcb9b221748>),
 (('Central African Republic', '2241813.8'),
  <pyspark.resultiterable.ResultIterable at 0x7fcb9b221b38>),
 (('Dominica', '11130355.8'),
  <pyspark.resultiterable.ResultIterable at 0x7fcb9b221128>),
 (('Dominican Republic', '254728306.1'),
  <pyspark.resultiterable.ResultIterable at 0x7fcb9b221e80>),
 (('El Salvador', '298939687.1'),
  <pyspark.resultiterable.ResultIterable at 0x7fcb9b2212e8>),
 (('Georgia', '144062401.5'),
  <pyspark.resultiterable.ResultIterable at 0x7fcb9b221358>),
 (('Grenada', '13740888.8'),
  <pyspark.resultiterable.ResultIterable at 0x7fcb9b221080>),
 (('Guatemala', '339822253.6'),
  <p

4. Media de los indicadores de deuda (tabla uno): DT.AMT.BLAT.CD, DT.DIS.BLAT.CD, DT.INT.BLAT.CD

In [ ]:
#ponemos cero a los nan vacio o null
#y despues a los otros los convertimos en tipo float
def tofloat(x):
        if x in ["", " ","nan","NULL"]: 
            return 0.0
        else:
            return float(x)

In [ ]:
#nos vamos a la columna dos (x[2])) y despues le tiramos la media
clean_beta_deuda.map(lambda x: tofloat(x[2])).mean()    

701125769.5846772

In [ ]:
#de la misma manera para vaias columns
clean_beta_deuda.map(lambda x: tofloat(x[3])).mean()   

5904868401.499194

5. Los 20 paises con DT.AMT.DLXF.CD más alto



In [ ]:
maximos20 = clean_beta_deuda.sortBy(lambda x: tofloat(x[3]), ascending = False).map(lambda x: (x[0], tofloat(x[3])))
#revisar como se componia esta linea

In [ ]:
# estan ordenados ya entonces pedimos los 20 primeros
maximos20.take(20)        

[('China', 96218620835.7),
 ('Brazil', 90041840304.1),
 ('Russian Federation', 66589761833.5),
 ('Turkey', 51555031005.8),
 ('South Asia', 48756295898.2),
 ('India', 31923507000.8),
 ('Indonesia', 30916112653.8),
 ('Kazakhstan', 27482093686.4),
 ('Mexico', 25218503927.0),
 ('Least developed countries: UN classification', 25197029299.4),
 ('IDA only', 20483289208.0),
 ('Romania', 14013783350.4),
 ('Colombia', 11985674438.7),
 ('Angola', 11067045628.1),
 ('Cameroon', 10404814960.2),
 ('"Venezuela', 9878659207.2),
 ('"Egypt', 9692114176.9),
 ('Lebanon', 9506919669.6),
 ('South Africa', 9474257551.9),
 ('Vietnam', 8873505909.2)]

 6. Pais con los datos informados todos los años.

In [ ]:
indix = clean_beta_desarrollo.map(lambda x: (x[0],1)).reduceByKey(lambda x,y: x+y).sortBy(lambda x: x[1], False)

#metemos contador y luego ordenamos descende
#revisar el como metemos la unidad adicional


In [ ]:
indix.take(56)
#malawi brinda 4 años de informacion, es el que mas

[('Malawi', 4),
 ('Botswana', 3),
 ('Guatemala', 3),
 ('Hungary', 3),
 ('Palau', 3),
 ('Russian Federation', 3),
 ('Switzerland', 3),
 ('Ukraine', 3),
 ('United States', 3),
 ('Uruguay', 3),
 ('Belarus', 3),
 ('Georgia', 3),
 ('Jamaica', 3),
 ('Japan', 3),
 ('Moldova', 3),
 ('New Zealand', 3),
 ('Singapore', 3),
 ('Spain', 3),
 ('Turkey', 3),
 ('United Kingdom', 3),
 ('"Bahamas', 3),
 ('Albania', 3),
 ('Australia', 3),
 ('Bhutan', 3),
 ('Brazil', 3),
 ('El Salvador', 3),
 ('Ireland', 3),
 ('Kazakhstan', 3),
 ('Kyrgyz Republic', 3),
 ('Peru', 3),
 ('Solomon Islands', 3),
 ('"Korea', 3),
 ('"Micronesia', 3),
 ('Colombia', 3),
 ('Indonesia', 3),
 ('Malaysia', 3),
 ('Tonga', 3),
 ('Zambia', 2),
 ('Jordan', 2),
 ('Papua New Guinea', 2),
 ('Seychelles', 2),
 ('Barbados', 2),
 ('Thailand', 2),
 ('San Marino', 2),
 ('Sri Lanka', 2),
 ('Belize', 1),
 ('Namibia', 1),
 ('St. Lucia', 1),
 ('Philippines', 1),
 ('St. Kitts and Nevis', 1),
 ('St. Vincent and the Grenadines', 1)]

7. Media anual de los distintos indicadores de desarrollo

In [ ]:
#filter + mean
a2014 = clean_beta_desarrollo.filter(lambda x: x[2] == '2014 [YR2014]')
a2014.map(lambda x: tofloat(x[3])).mean() 

103442758732953.22

In [ ]:
a2015 = clean_beta_desarrollo.filter(lambda x: x[2] == '2015 [YR2015]')
a2015.map(lambda x: tofloat(x[3])).mean() 

131824403915902.5

In [ ]:
a2016 = clean_beta_desarrollo.filter(lambda x: x[2] == '2016 [YR2016]')
a2016.map(lambda x: tofloat(x[3])).mean() 

158813821548065.94

In [ ]:
#Ahora le metemos otra vez filter  con el porcentaje del pib... y tiramos la media
pib2014 = clean_beta_desarrollo.filter(lambda x: x[2] == '2014 [YR2014]')
pib2014.map(lambda x: tofloat(x[4])).mean() 

56.84391434438437

In [ ]:
pib2015 = clean_beta_desarrollo.filter(lambda x: x[2] == '2015 [YR2015]')
pib2015.map(lambda x: tofloat(x[4])).mean() 

58.36559834488035

In [ ]:
pib2016 = clean_beta_desarrollo.filter(lambda x: x[2] == '2016 [YR2016]')
pib2016.map(lambda x: tofloat(x[4])).mean() 

58.796949624008356

In [ ]:
#Ahora para birth rate, crude (per 1,00 people) y tirmos media igual 

br_2014 = clean_beta_desarrollo.filter(lambda x: x[2] == '2014 [YR2014]')
br_2014.map(lambda x: tofloat(x[5])).mean() 

17.582620000000002

In [ ]:
br_2015 = clean_beta_desarrollo.filter(lambda x: x[2] == '2015 [YR2015]')
br_2015.map(lambda x: tofloat(x[5])).mean() 

16.798477272727272

In [ ]:
br_2016 = clean_beta_desarrollo.filter(lambda x: x[2] == '2016 [YR2016]')
br_2016.map(lambda x: tofloat(x[5])).mean() 

16.144333333333336

In [ ]:
#malawi es la que brinda los datos estos que como comprobamos antes era la unica con tantos años (4)
a2017 = clean_beta_desarrollo.filter(lambda x: x[2] == '2017 [YR2017]')
a2017.map(lambda x: tofloat(x[3])).mean()

2239858619962.69

In [ ]:
pib2017 = clean_beta_desarrollo.filter(lambda x: x[2] == '2017 [YR2017]')
pib2017.map(lambda x: tofloat(x[4])).mean() 

48.659564238322

In [ ]:
br_2017 = clean_beta_desarrollo.filter(lambda x: x[2] == '2017 [YR2017]')
br_2017.map(lambda x: tofloat(x[5])).mean() 

34.593

8. Podrías decirme el total de deuda acumulada DT.AMT.MLAT.CD por los 10 países con un valor en media menor de SP.DYN.CBRT.IN

In [ ]:
#falta la 9 también